In [26]:
# from langchain_core.tools import tool

# @tool
# def multiply(a: int, b: int) -> int:
#     """Multiply two numbers."""
#     return a * b

In [27]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="The search query.")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Perform a search operation."""
    return f"Results for {query}"

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
import os
os.environ['GOOGLE_API_KEY']=""
llm=GoogleGenerativeAI(model="gemini-1.5-flash",temperature="0.60")

In [29]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

c:\Users\Admin\anaconda3\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [45]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


In [46]:
from langchain.agents import initialize_agent, AgentType,create_react_agent

# List of tools
tools = [multiply, search]

# Initialize the agent
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)

In [47]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [48]:
agent_executor.invoke({"input":"What is 2 * 5"})



> Entering new AgentExecutor chain...
Thought: I can use the multiplication-tool to calculate this.
Action: multiplication-tool
Action Input: a=2, b=5

ValidationError: 2 validation errors for CalculatorInput
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a=2, b=5', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
b
  Field required [type=missing, input_value={'a': 'a=2, b=5'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing